In [ ]:
import matplotlib.pyplot as plt

from data.NSF_data import Argo1_NSF
from ops.visibility2D import get_range_img_coords
import torch
import numpy as np
from vis.deprecated_vis import *
from data.PATHS import TMP_VIS_PATH

nsf_dataloader = Argo1_NSF()
for batch_id, data in enumerate(nsf_dataloader):
    pc1, pc2, gt_flow = data
    break

pc = pc1[0].numpy()

yaw, pitch, depth = get_range_img_coords(pc, VOF=50, HOF=1024)
depth_pc = np.linalg.norm(pc, axis=1)

# visualize_points3D(pc, depth_pc)

In [19]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=1., min_samples=5)

margin = 5
# cylinder = np.stack([yaw, pitch, depth_pc * 10], axis=1) # normilized yaw, pitch increment =1
# dbscan.fit(cylinder)  # not good?

dbscan.fit(pc)
labels = dbscan.labels_


visualize_points3D(pc[labels==0], pc[labels==0, 2])

# Clusters separated by visibility ray as line
    - view it as linear classification of point instances
    - computation as Linear layer

    - toy example [x]
    - define which rays to take []
    - 2D so far? how to get
    - define suitable depth []

In [ ]:
pc_clu = pc[labels==0]
sim_ray = np.array(([pc_clu.mean(0), np.array([0,0,0])]))
print(sim_ray)

a = sim_ray[0,1] - sim_ray[1,1]
b = sim_ray[1,0] - sim_ray[0,0]
c = a * sim_ray[0,0] + b * sim_ray[0, 1]

plt.close()
plt.plot(sim_ray[:,0], sim_ray[:,1], 'r-')
plt.plot(pc_clu[:,0], pc_clu[:,1], 'y.')
plt.savefig(TMP_VIS_PATH + '/prd.png')


In [30]:
line_eq = np.array((a,b,c))
distance = pc_clu @ line_eq
instances = distance > 0
# visualize_points3D(pc_clu, instances)

plt.plot(pc_clu[instances, 0], pc_clu[instances, 1], 'y.')
plt.plot(pc_clu[instances==False, 0], pc_clu[instances==False, 1], 'g.')
plt.savefig(TMP_VIS_PATH + '/prd.png')
